# 🚀 Sequential Agent Workflow (Notebook Version)

This Colab-ready notebook mirrors your Python script and runs a **sequential agentic workflow** using the Microsoft Agent Framework (`agent_framework`).  
It sets up two domain agents — a **writer** and a **reviewer** — and streams the final conversation.

> **Auth note (Colab-friendly):** Your original script uses `AzureCliCredential()`. In Colab, the simplest path is to use **Azure OpenAI API key auth** via environment variables. This notebook supports both; API key is the default.


## 1) Install dependencies

- Tries to import `agent_framework`.  
- If unavailable, it **auto-installs** from the official GitHub repo as a fallback.
- Installs `azure-identity` for AAD auth (optional if using API key).


In [2]:
! pip install agent-framework-core  -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.7 MB/s eta 0:00:00


In [3]:
# If you're running in Colab, uncomment the next line to ensure a clean pip state
# %pip install -U pip

import sys, subprocess, importlib

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

# Try to import agent_framework; if not found, install from GitHub
try:
    import agent_framework  # noqa: F401
except Exception:
    print("agent_framework not found. Installing from GitHub repo...")
    pip_install(["git+https://github.com/microsoft/ai-agents-for-beginners.git#subdirectory=14-microsoft-agent-framework"])
    # Retry import
    import agent_framework  # noqa: F401

# Azure identity is useful if you want to use AAD (AzureCliCredential / DefaultAzureCredential)
try:
    import azure.identity  # noqa: F401
except Exception:
    pip_install(["azure-identity"])

## 2) Configure Azure OpenAI credentials

Choose **one** of the two auth paths below:

### A) API Key (recommended for Colab)
Set the following:
- `AZURE_OPENAI_API_KEY` – your Azure OpenAI key  
- `AZURE_OPENAI_ENDPOINT` – e.g. `https://YOUR_RESOURCE_NAME.openai.azure.com/`  
- One of:
  - `AZURE_OPENAI_DEPLOYMENT` (deployment name of your chat model), **or**
  - `AZURE_OPENAI_MODEL` (model name if your framework version supports it)

### B) AAD (Azure CLI / Entra ID)
- Run `az login` locally or in your environment and ensure your identity has access.
- The code will use `AzureCliCredential()` if you flip the `USE_AAD` flag below.


In [6]:
# import os

# # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# # Fill these in for API Key auth (recommended in Colab):
# os.environ.setdefault("AZURE_OPENAI_API_KEY",    "YOUR_API_KEY_HERE")
# os.environ.setdefault("AZURE_OPENAI_ENDPOINT",   "https://YOUR_RESOURCE_NAME.openai.azure.com/")
# # Set at least one of the following depending on your setup:
# os.environ.setdefault("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini")  # example deployment name
# # os.environ.setdefault("AZURE_OPENAI_MODEL",   "gpt-4o-mini")   # alternative if your framework supports a model var
# # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# # Toggle this if you want to try Azure AD auth instead of API key (not typical in Colab)
USE_AAD = False

# print("Endpoint:", os.environ.get("AZURE_OPENAI_ENDPOINT"))
# print("Using AAD credential:", USE_AAD)

In [4]:
from google.colab import userdata
AZURE_OPENAI_API_KEY = userdata.get('GITHUB_TOKEN')
AZURE_OPENAI_ENDPOINT = userdata.get('GITHUB_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = userdata.get('GITHUB_MODEL_ID')

## 3) Define and run the workflow

This mirrors your original script with small notebook-friendly tweaks:
- Prefer **API key auth** by default.
- Uses **top-level `await`** for smooth async execution in notebooks.


In [10]:
import asyncio
from typing import cast

from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent

# We'll try API key-based client first, with AAD as an optional fallback.
# Different versions of the framework may expose AzureOpenAIChatClient in slightly different places.
# We'll handle both common import paths gracefully.
try:
    from agent_framework.azure import AzureOpenAIChatClient
except Exception:
    # Some versions expose clients under agent_framework.clients
    from agent_framework.clients.azure import AzureOpenAIChatClient  # type: ignore

# Optional AAD cred if you flip the flag
aad_credential = None
if USE_AAD:
    try:
        from azure.identity import AzureCliCredential
        aad_credential = AzureCliCredential()
    except Exception as e:
        print("azure-identity not available or Azure CLI not configured:", e)

def make_client():
    """Build an AzureOpenAIChatClient using either API key env vars or AAD."""
    if not USE_AAD:
        # API key path — most reliable for Colab
        # endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
        endpoint = AZURE_OPENAI_ENDPOINT
        # api_key = os.environ.get("AZURE_OPENAI_API_KEY")
        api_key = AZURE_OPENAI_API_KEY
        deployment = AZURE_OPENAI_DEPLOYMENT
        if not endpoint or not api_key or not deployment:
            raise RuntimeError("Missing endpoint, api key, or deployment name.")
        return AzureOpenAIChatClient(
            endpoint=endpoint,
            api_key=api_key,
            deployment_name=deployment   # <-- crucial
        )
    else:
        # AAD path (requires `az login` in your environment)
        if aad_credential is None:
            raise RuntimeError("AAD credential not available. Install azure-identity and run az login.")
        return AzureOpenAIChatClient(credential=aad_credential)

async def main() -> None:
    chat_client = make_client()

    # If your framework needs an explicit deployment/model, you can set it here
    # via environment variables or pass during agent creation (varies by version).
    writer = chat_client.create_agent(
        instructions=("You are a concise copywriter. Provide a single, punchy marketing sentence based on the prompt."),
        name="writer",
    )
    reviewer = chat_client.create_agent(
        instructions=("You are a thoughtful reviewer. Give brief feedback on the previous assistant message."),
        name="reviewer",
    )

    # Build sequential workflow: writer -> reviewer
    workflow = SequentialBuilder().participants([writer, reviewer]).build()

    # Run and collect outputs
    outputs: list[list[ChatMessage]] = []
    async for event in workflow.run_stream("Write a tagline for a budget-friendly eBike."):
        if isinstance(event, WorkflowOutputEvent):
            outputs.append(cast(list[ChatMessage], event.data))

    if outputs:
        print("===== Final Conversation =====")
        for i, msg in enumerate(outputs[-1], start=1):
            name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
            print(f"{'-' * 60}\n{i:02d} [{name}]\n{msg.text}")

# In modern IPython/Colab, top-level await works:
await main()

===== Final Conversation =====
------------------------------------------------------------
01 [user]
Write a tagline for a budget-friendly eBike.
------------------------------------------------------------
02 [writer]
"Ride Green, Spend Less: Your Affordable eBike Adventure Awaits!"
------------------------------------------------------------
03 [reviewer]
This tagline effectively captures the budget-friendly aspect of the eBike while also appealing to environmentally conscious consumers. The phrase "Ride Green" promotes sustainability, and "Spend Less" highlights affordability. The added touch of "Your Affordable eBike Adventure Awaits" encourages excitement and a sense of exploration, making it an inviting invitation for potential customers. Overall, it’s a strong and engaging tagline!


## ✅ Troubleshooting

- **`agent_framework` import fails**: The install cell auto-installs from GitHub. If your environment lacks build tools, try re‑running the cell.
- **Auth errors**:
  - For **API key**: confirm `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, and your **deployment/model** are correct and enabled.
  - For **AAD**: ensure `azure-identity` is installed and you've run `az login` in an environment with access.
- **Model/Deployment missing**: Some versions require an explicit `deployment` or `model`. Set `AZURE_OPENAI_DEPLOYMENT` or `AZURE_OPENAI_MODEL`, or check the framework docs for your version.
